In [2]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
diamonds = sns.load_dataset("diamonds")
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [5]:
diamonds.shape

(53940, 10)

In [6]:
diamonds.describe

<bound method NDFrame.describe of        carat        cut color clarity  depth  table  price     x     y     z
0       0.23      Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1       0.21    Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2       0.23       Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3       0.29    Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4       0.31       Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75
...      ...        ...   ...     ...    ...    ...    ...   ...   ...   ...
53935   0.72      Ideal     D     SI1   60.8   57.0   2757  5.75  5.76  3.50
53936   0.72       Good     D     SI1   63.1   55.0   2757  5.69  5.75  3.61
53937   0.70  Very Good     D     SI1   62.8   60.0   2757  5.66  5.68  3.56
53938   0.86    Premium     H     SI2   61.0   58.0   2757  6.15  6.12  3.74
53939   0.75      Ideal     D     SI2   62.2   55.0   2757  5.83  5.87  3.64

[53940 rows x 10 columns]>

In [7]:
diamonds.describe(exclude=np.number)

,cut,color,clarity
count,53940,53940,53940
unique,5,7,8
top,Ideal,G,SI1
freq,21551,11292,13065


In [8]:
from sklearn.model_selection import train_test_split

# Extract feature and target arrays
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

In [9]:
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to Pandas category
for col in cats:
   X[col] = X[col].astype('category')

In [10]:
X.dtypes

carat       float64
cut        category
color      category
clarity    category
depth       float64
table       float64
x           float64
y           float64
z           float64
dtype: object

In [11]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
import xgboost as xgb

# Create regression matrices
dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

## Training

The chosen objective function and any other hyperparameters of XGBoost should be specified in a dictionary, which by convention should be called params:

In [13]:
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

Inside this initial params, we are also setting tree_method to gpu_hist, which enables GPU acceleration.

Now, we set another parameter called num_boost_round, which stands for number of boosting rounds. Internally, XGBoost minimizes the loss function RMSE in small incremental rounds (more on this later). This parameter specifies the amount of those rounds. The ideal number of rounds is found through hyperparameter tuning. For now, we will just set it to 100:

In [14]:
n = 100
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
)

## Evaluation

During the boosting rounds, the model object has learned all the patterns of the training set it possibly can. Now, we must measure its performance by testing it on unseen data. That's where our dtest_reg DMatrix comes into play:

In [15]:
from sklearn.metrics import mean_squared_error

preds = model.predict(dtest_reg)

This step of the process is called model evaluation (or inference). Once you generate predictions with predict, you pass them inside mean_squared_error function of Sklearn to compare against y_test:

In [16]:
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE of the base model: {rmse:.3f}")

RMSE of the base model: 555.607


We’ve got a base score ~543$, which was the performance of a base model with default parameters. There are two ways we can improve it— by performing cross-validation and hyperparameter tuning. But before that, let’s see a quicker way of evaluating XGBoost models.

## Using Validation Sets During Training

Training a machine learning model is like launching a rocket into space. You can control everything about the model up to the launch, but once it does, all you can do is stand by and wait for it to finish.

But the problem with our current training process is that we can’t even watch where the model is going. To solve this, we will use evaluation arrays that allow us to see model performance as it gets improved incrementally across boosting rounds.

First, let’s set up the parameters again:

In [17]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

Next, we create a list of two tuples that each contain two elements. The first element is the array for the model to evaluate, and the second is the array’s name.

In [18]:
evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

When we pass this array to the evals parameter of xgb.train, we will see the model performance after each boosting round:

In [19]:
model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
)

[0]	train-rmse:2874.29379	validation-rmse:2817.38773
[1]	train-rmse:2092.07711	validation-rmse:2054.73630
[2]	train-rmse:1549.52687	validation-rmse:1526.30592
[3]	train-rmse:1184.46798	validation-rmse:1174.90119
[4]	train-rmse:941.09127	validation-rmse:943.28272
[5]	train-rmse:784.58014	validation-rmse:796.09651
[6]	train-rmse:685.75110	validation-rmse:705.22245
[7]	train-rmse:624.67281	validation-rmse:653.32563
[8]	train-rmse:584.19599	validation-rmse:620.30404
[9]	train-rmse:558.77667	validation-rmse:599.24504
[10]	train-rmse:543.85303	validation-rmse:586.99790
[11]	train-rmse:531.92694	validation-rmse:578.68120
[12]	train-rmse:523.08456	validation-rmse:571.73527
[13]	train-rmse:515.67753	validation-rmse:567.19913
[14]	train-rmse:510.77594	validation-rmse:564.66402
[15]	train-rmse:506.68519	validation-rmse:563.21547
[16]	train-rmse:502.96796	validation-rmse:561.80880
[17]	train-rmse:498.90184	validation-rmse:560.36561
[18]	train-rmse:492.74859	validation-rmse:558.46274
[19]	train-rms

In real-world projects, you usually train for thousands of boosting rounds, which means that many rows of output. To reduce them, you can use the verbose_eval parameter, which forces XGBoost to print performance updates every vebose_eval rounds:

In [20]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 100

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=10 # Every ten rounds
)

[0]	validation-rmse:2817.38773	train-rmse:2874.29379
[10]	validation-rmse:586.99790	train-rmse:543.85303
[20]	validation-rmse:556.44229	train-rmse:487.42071
[30]	validation-rmse:554.68339	train-rmse:460.86396
[40]	validation-rmse:552.62130	train-rmse:444.03762
[50]	validation-rmse:553.50718	train-rmse:430.07110
[60]	validation-rmse:555.44368	train-rmse:418.57995
[70]	validation-rmse:555.06703	train-rmse:406.77489
[80]	validation-rmse:555.00800	train-rmse:394.18070
[90]	validation-rmse:555.74725	train-rmse:382.65353
[99]	validation-rmse:555.60692	train-rmse:373.74308


## XGBoost Early Stopping

By now, you must have realized how important boosting rounds are. Generally, the more rounds there are, the more XGBoost tries to minimize the loss. But this doesn’t mean the loss will always go down. Let’s try with 5000 boosting rounds with the verbosity of 500:

In [21]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 5000

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=250
)

[0]	validation-rmse:2817.38773	train-rmse:2874.29379
[250]	validation-rmse:561.30944	train-rmse:276.85755
[500]	validation-rmse:563.29248	train-rmse:197.72375
[750]	validation-rmse:568.62872	train-rmse:150.03843
[1000]	validation-rmse:572.18689	train-rmse:121.69016
[1250]	validation-rmse:574.18885	train-rmse:99.34243
[1500]	validation-rmse:576.42698	train-rmse:84.17960
[1750]	validation-rmse:578.10716	train-rmse:71.14960
[2000]	validation-rmse:578.58142	train-rmse:60.88001
[2250]	validation-rmse:579.16086	train-rmse:52.80762
[2500]	validation-rmse:579.96944	train-rmse:46.47975
[2750]	validation-rmse:580.51823	train-rmse:41.25980
[3000]	validation-rmse:580.78514	train-rmse:36.59124
[3250]	validation-rmse:581.21700	train-rmse:32.91052
[3500]	validation-rmse:581.36774	train-rmse:29.47363
[3750]	validation-rmse:581.61295	train-rmse:26.40702
[4000]	validation-rmse:581.86083	train-rmse:23.90580
[4250]	validation-rmse:581.89800	train-rmse:21.68452
[4500]	validation-rmse:582.07031	train-rmse:1

We get the lowest loss before round 500. After that, even though training loss keeps going down, the validation loss (the one we care about) keeps increasing.

When given an unnecessary number of boosting rounds, XGBoost starts to overfit and memorize the dataset. This, in turn, leads to validation performance drop because the model is memorizing instead of generalizing.

Remember, we want the golden middle: a model that learned just enough patterns in training that it gives the highest performance on the validation set. So, how do we find the perfect number of boosting rounds, then?

We will use a technique called early stopping. Early stopping forces XGBoost to watch the validation loss, and if it stops improving for a specified number of rounds, it automatically stops training.

This means we can set as high a number of boosting rounds as long as we set a sensible number of early stopping rounds.

For example, let’s use 10000 boosting rounds and set the early_stopping_rounds parameter to 50. This way, XGBoost will automatically stop the training if validation loss doesn't improve for 50 consecutive rounds.

In [23]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 10000

evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]


model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=n,
   evals=evals,
   verbose_eval=250,
   # activate early stopping
   early_stopping_rounds=50
)

[0]	validation-rmse:2817.38773	train-rmse:2874.29379
[250]	validation-rmse:561.30944	train-rmse:276.85755
[500]	validation-rmse:563.29248	train-rmse:197.72375
[750]	validation-rmse:568.62872	train-rmse:150.03843
[1000]	validation-rmse:572.18689	train-rmse:121.69016
[1250]	validation-rmse:574.18885	train-rmse:99.34243
[1500]	validation-rmse:576.42698	train-rmse:84.17960
[1750]	validation-rmse:578.10716	train-rmse:71.14960
[2000]	validation-rmse:578.58142	train-rmse:60.88001
[2250]	validation-rmse:579.16086	train-rmse:52.80762
[2500]	validation-rmse:579.96944	train-rmse:46.47975
[2750]	validation-rmse:580.51823	train-rmse:41.25980
[3000]	validation-rmse:580.78514	train-rmse:36.59124
[3250]	validation-rmse:581.21700	train-rmse:32.91052
[3500]	validation-rmse:581.36774	train-rmse:29.47363
[3750]	validation-rmse:581.61295	train-rmse:26.40702
[4000]	validation-rmse:581.86083	train-rmse:23.90580
[4250]	validation-rmse:581.89800	train-rmse:21.68452
[4500]	validation-rmse:582.07031	train-rmse:1

## XGBoost Cross-Validation

In cross-validation, we still have two sets: training and testing. While the test set waits in the corner, we split the training into 3, 5, 7, or k splits or folds. Then, we train the model k times. Each time, we use k-1 parts for training and the final kth part for validation. This process is called k-fold cross-validation. After all folds are done, we can take the mean of the scores as the final, most realistic performance of the model. Let’s perform this process in code using the cv function of XGB:

In [24]:
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}
n = 1000

results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=n,
   nfold=5,
   early_stopping_rounds=20
)

In [25]:
results.head()

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,2874.530912,9.576510,2877.437274,37.093540
1,2089.327469,8.317290,2094.021636,24.828795
2,1550.617973,5.223297,1558.386252,18.540267
3,1183.812759,5.193420,1195.032441,13.471580
4,941.203113,4.539805,958.728828,9.479449


It has the same number of rows as the number of boosting rounds. Each row is the average of all splits for that round. So, to find the best score, we take the minimum of the test-rmse-mean column:

In [26]:
best_rmse = results['test-rmse-mean'].min()

Note that this method of cross-validation is used to see the true performance of the model. Once satisfied with its score, you must retrain it on the full data before deployment.

## XGBoost Classification

Building an XGBoost classifier is as easy as changing the objective function; the rest can stay the same.

The two most popular classification objectives are:

binary:logistic - binary classification (the target contains only two classes, i.e., cat or dog)
multi:softprob - multi-class classification (more than two classes in the target, i.e., apple/orange/banana)
Performing binary and multi-class classification in XGBoost is almost identical, so we will go with the latter. Let’s prepare the data for the task first.

We want to predict the cut quality of diamonds given their price and their physical measurements. So, we will build the feature/target arrays accordingly:

In [27]:
from sklearn.preprocessing import OrdinalEncoder

X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]

# Encode y to numeric
y_encoded = OrdinalEncoder().fit_transform(y)

# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()

# Convert to pd.Categorical
for col in cats:
   X[col] = X[col].astype('category')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)

The only difference is that since XGBoost only accepts numbers in the target, we are encoding the text classes in the target with OrdinalEncoder of Sklearn. Now, we build the DMatrices…

In [28]:
# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

…and set the objective to multi:softprob. This objective also requires the number of classes to be set by us:

In [29]:
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 5}
n = 1000

results = xgb.cv(
   params, dtrain_clf,
   num_boost_round=n,
   nfold=5,
   metrics=["mlogloss", "auc", "merror"],
)

During cross-validation, we are asking XGBoost to watch three classification metrics which report model performance from three different angles. Here is the result:

In [31]:
results.keys()

Index(['train-mlogloss-mean', 'train-mlogloss-std', 'train-auc-mean',
       'train-auc-std', 'train-merror-mean', 'train-merror-std',
       'test-mlogloss-mean', 'test-mlogloss-std', 'test-auc-mean',
       'test-auc-std', 'test-merror-mean', 'test-merror-std'],
      dtype='object')

To see the best AUC score, we take the maximum of test-auc-mean column:

In [32]:
results['test-auc-mean'].max()

0.9402233623451636

## XGBoost Native vs. XGBoost Sklearn

So far, we have been using the native XGBoost API, but its Sklearn API is pretty popular as well.

Sklearn is a vast framework with many machine learning algorithms and utilities and has an API syntax loved by almost everyone. Therefore, XGBoost also offers XGBClassifier and XGBRegressor classes so that they can be integrated into the Sklearn ecosystem (at the loss of some of the functionality).

If you want to only use the Scikit-learn API whenever possible and only switch to native when you need access to extra functionality, there is a way.

After training the XGBoost classifier or regressor, you can convert it using the get_booster method:

In [33]:
import xgboost as xgb

# Train a model using the scikit-learn API
xgb_classifier = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', tree_method='hist', eta=0.1, max_depth=3, enable_categorical=True)
xgb_classifier.fit(X_train, y_train)

# Convert the model to a native API model
model = xgb_classifier.get_booster()